ATTENTION: This notebook is based on old mock data. It is in this git repository for reference. However this material may be outdated for future users who are reading this.

# ML Project - Sean Slavich - Building a Model

In this notebook, we will focus on a model that can predict acceptance rate of an interview on behalf of an employee. This model will be integrated with QuickSight, which will allow the dashboard to be dynamically updated over time.

In [159]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [163]:
import seaborn as sns
import matplotlib.pyplot as plt 

bucket = 'ml-analytics-data-2022/Data'
bucket2 = 'hiring-curated-bucket/master-table/test_csv/date_col=2022-07-21-21-19-00.262'

skill_data_key = 'Employee_Skills.csv'
skill_data_location = 's3://{}/{}'.format(bucket, skill_data_key)

interviewee_data_key = 'Interviewee_Hiring.csv'
interviewee_data_location = 's3://{}/{}'.format(bucket, interviewee_data_key)

interviewer_data_key = 'Interviewer_Hiring.csv'
interviewer_data_location = 's3://{}/{}'.format(bucket, interviewer_data_key)

scheduling_data_key = 'Scheduling.csv'
scheduling_data_location = 's3://{}/{}'.format(bucket, scheduling_data_key)

voop_data_key = 'Voop_Data.csv'
voop_data_location = 's3://{}/{}'.format(bucket, voop_data_key)

master_data_key = 'Master_df.csv'
master_data_location = 's3://{}/{}'.format(bucket, master_data_key)

master_data_key_new = 'part-00000-87310c10-d1ce-415b-97ba-9db46e692b3c.c000.csv'
master_data_location_new = 's3://{}/{}'.format(bucket2, master_data_key_new)

emp_skills = pd.read_csv(skill_data_location)
interviewee_hiring = pd.read_csv(interviewee_data_location)
interviewer_hiring = pd.read_csv(interviewer_data_location)
scheduling = pd.read_csv(scheduling_data_location)
voop_data = pd.read_csv(voop_data_location)
master_df = pd.read_csv(master_data_location)
master_df_new = pd.read_csv(master_data_location_new)

We cannot use the master_df table here as these joins were performed on employee alias. We will rejoin data and only include features needed to be tested for the linear learner algorithm.

In [4]:
master_df

,Employee_Alias,Count_of_Scheduled_Interviews,Count_of_Accepted_Interviews,Skill,Skill_Rating,Last_Updated_Emp_Skills,Interview_ID,Interviewer_Name,Candidate_ICMS_ID,Interviewer_Role,...,Candidate_Name,Decision,Duration,Level,Interview_Type,Communication_Rating,Leadership_Rating,Technical_Rating,Interest_Rating,Last_Updated_Voop
0,aantukit,1,1,ML/AI - Forecast,2,2021-08-30T13:58:25Z,39367780,Abbe Antuk,71588916,Technical Account Manager,...,Carmelia Acosta,Strongly Inclined,90,L5,Phone Screen,6,4,1,2,2022-03-07T05:59:47Z
1,aarstingall1z,1,1,Mobile - Pinpoint,1,2022-05-14T13:05:19Z,78838328,Anastasia Arstingall,51740953,Operations Manager,...,Ernesto Dods,Not Inclined,45,L8,Phone Screen,3,6,6,6,2021-07-16T01:54:29Z
2,aaustenh8,1,1,ML/AI - Rekognition,2,2022-06-02T07:47:43Z,69014079,Angeli Austen,38560249,Delivery Practice Manager,...,Fidelio Varnals,Not Inclined,30,L6,Phone Screen,1,4,1,3,2022-06-16T10:25:23Z
3,ababbsog,1,1,Compute and HPC - EC2 (Elastic Compute Cloud),5,2021-09-25T06:43:48Z,29095642,Aggie Babbs,67393166,Cloud Support Engineer,...,Anastasie Fessier,Strong No Hire,30,L4,Bar Raiser,6,4,3,6,2021-08-28T10:47:21Z
4,abagworthhs,1,1,ML/AI - SageMaker,5,2022-04-18T09:12:25Z,68439136,Aubrey Bagworth,18400364,Technical Account Manager,...,Karyl Lafflina,Strongly Inclined,30,L6,Loop,2,6,3,1,2022-05-12T22:12:31Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,yroberti93,1,1,Analytics - Lake Formation,1,2021-07-26T00:56:14Z,93932350,Yale Roberti,28959740,Software Engineer,...,Evy Truelove,Strong No Hire,45,L9,Phone Screen,5,3,4,4,2021-09-22T22:19:41Z
996,zellingsi,1,1,Database - RDS SQL Server,1,2021-10-21T16:49:04Z,20784972,Zach Ellings,78453163,Data Engineer,...,Eb Hawkswell,Not Inclined,90,L4,Loop,6,4,4,2,2022-03-27T12:58:29Z
997,zespadas35,1,1,ML/AI - SageMaker,4,2021-08-12T17:22:27Z,60227282,Zena Espadas,20919242,Cloud Architect,...,Kerby Edeler,Inclined,45,L7,Phone Screen,5,6,4,5,2022-04-05T08:35:25Z
998,zjouanny71,1,1,ML/AI - Rekognition,2,2021-10-28T14:26:51Z,4884977,Zsazsa Jouanny,95392950,Delivery Practice Manager,...,Vinnie Burfoot,Strong No Hire,30,L7,Loop,3,5,6,4,2021-10-24T05:27:24Z


In [105]:
voop_data

,Interview_ID,Employee_Alias,Date,Time,Candidate_Name,Decision,Duration,Level,Interview_Type,Communication_Rating,Leadership_Rating,Technical_Rating,Interest_Rating,Last_Updated
0,36977658,ldelepine0,4/11/22,13:00,Agnese Normanton,Strong No Hire,45,L9,Bar Raiser,4,5,6,2,2021-10-07T10:29:57Z
1,67484217,aharral1,11/3/21,9:00,Davey Abrams,Inclined,45,L8,Loop,6,5,6,1,2022-02-04T23:26:12Z
2,84630967,sdryburgh2,10/27/21,12:00,Marrissa Langhor,Inclined,60,L6,Phone Screen,3,5,2,6,2021-12-08T16:11:52Z
3,9888727,chugonet3,5/26/22,12:00,Aurora Ireland,Inclined,90,L6,Phone Screen,2,2,3,6,2021-08-08T16:13:47Z
4,60055003,cfawthrop4,5/13/22,15:00,Debora Greenall,Strong No Hire,30,L6,Loop,6,3,3,1,2022-02-21T21:28:31Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,14556751,tvedntyevrn,8/20/21,17:00,Burgess Baumer,Strongly Inclined,30,L5,Bar Raiser,3,1,4,4,2021-12-15T02:44:08Z
996,83377630,afrakero,8/21/21,14:00,Jocelyn Howman,Strong No Hire,90,L5,Loop,1,4,5,6,2021-10-06T12:35:29Z
997,40462641,ttilleyrp,8/4/21,10:00,Ignacio Chivers,Not Inclined,30,L9,Loop,3,6,3,2,2021-11-09T00:25:06Z
998,78822341,bwadwellrq,1/19/22,9:00,Stillman Sproson,Not Inclined,90,L8,Phone Screen,4,5,3,6,2021-11-15T02:10:53Z


In [6]:
interviewer_hiring

,Interview_ID,Employee_Alias,Interviewer_Name,Candidate_Name,Candidate_ICMS_ID,Role,Type,Level,Preferred_Time,Preferred_Day,Last_Updated
0,51092998,ldelepine0,Linea Delepine,Agnese Normanton,7203708,Delivery Practice Manager,Contract,L4,10:00,Thursday,2021-08-20T23:11:27Z
1,99241636,aharral1,Arielle Harral,Davey Abrams,15918070,Software Development Engineer,Part-time,L7,14:00,Wednesday,2022-02-21T06:47:12Z
2,64517874,sdryburgh2,Shayne Dryburgh,Marrissa Langhor,67232318,Marketing Manager,Full-time,L6,14:00,Thursday,2021-12-28T15:00:16Z
3,75309134,chugonet3,Claudetta Hugonet,Aurora Ireland,82904801,Security Engineer,Contract,L9,16:00,Monday,2022-05-10T13:58:46Z
4,31410407,cfawthrop4,Cyrille Fawthrop,Debora Greenall,76196522,Cloud Support Engineer,Part-time,L5,12:00,Friday,2022-03-14T11:33:48Z
...,...,...,...,...,...,...,...,...,...,...,...
995,79716724,tvedntyevrn,Terrie Vedntyev,Burgess Baumer,39023047,Data Scientist,Part-time,L7,16:00,Tuesday,2021-06-21T06:27:20Z
996,36886778,afrakero,Alvan Frake,Jocelyn Howman,40070522,Solutions Architect,Internship,L8,14:00,Wednesday,2022-05-26T18:57:19Z
997,51097937,ttilleyrp,Torry Tilley,Ignacio Chivers,40720089,Product Manager,Contract,L4,14:00,Tuesday,2021-07-10T01:44:33Z
998,88855859,bwadwellrq,Bryn Wadwell,Stillman Sproson,74769886,EC2 Systems Engineer,Part-time,L9,14:00,Wednesday,2022-05-23T06:58:31Z


In [7]:
interviewee_hiring

,Interview_ID,Candidate_Name,Candidate_ICMS_ID,Role,Type,Level,Preferred_Time,Preferred_Day,Last_Updated
0,51092998,Agnese Normanton,7203708,EC2 Systems Engineer,Contract,L9,15:00,Tuesday,2022-02-04T16:18:37Z
1,99241636,Davey Abrams,15918070,Professional Services Consultant,Contract,L8,13:00,Monday,2022-03-20T01:04:55Z
2,64517874,Marrissa Langhor,67232318,EC2 Systems Engineer,Part-time,L6,14:00,Thursday,2022-03-18T12:56:21Z
3,75309134,Aurora Ireland,82904801,Machine Learning Engineer,Internship,L6,12:00,Tuesday,2021-11-26T16:06:23Z
4,31410407,Debora Greenall,76196522,Cloud Architect,Full-time,L6,16:00,Wednesday,2022-02-20T12:24:35Z
...,...,...,...,...,...,...,...,...,...
995,79716724,Burgess Baumer,39023047,Delivery Practice Manager,Part-time,L5,12:00,Monday,2022-05-08T22:34:48Z
996,36886778,Jocelyn Howman,40070522,Marketing Manager,Full-time,L5,10:00,Friday,2021-07-01T16:28:56Z
997,51097937,Ignacio Chivers,40720089,Security Engineer,Internship,L9,11:00,Monday,2022-04-22T09:03:40Z
998,88855859,Stillman Sproson,74769886,Software Engineer,Contract,L8,13:00,Friday,2022-05-19T20:51:46Z


# Drop Irrelevant Columns

In [13]:
int_hire_drop = interviewer_hiring.drop(columns=['Interviewer_Name', 'Candidate_Name', 'Candidate_ICMS_ID', 'Last_Updated', 'Type'])
int_hire_drop

,Interview_ID,Employee_Alias,Role,Level,Preferred_Time,Preferred_Day,Preferred_Day_Match,Preferred_Time_Match,Role_Match,Accepted_or_Not?
0,51092998,ldelepine0,Delivery Practice Manager,L4,10:00,Thursday,0,0,0,0
1,99241636,aharral1,Software Development Engineer,L7,14:00,Wednesday,0,0,0,1
2,64517874,sdryburgh2,Marketing Manager,L6,14:00,Thursday,1,1,0,0
3,75309134,chugonet3,Security Engineer,L9,16:00,Monday,0,0,0,0
4,31410407,cfawthrop4,Cloud Support Engineer,L5,12:00,Friday,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
995,79716724,tvedntyevrn,Data Scientist,L7,16:00,Tuesday,0,0,0,0
996,36886778,afrakero,Solutions Architect,L8,14:00,Wednesday,0,0,0,0
997,51097937,ttilleyrp,Product Manager,L4,14:00,Tuesday,0,0,0,1
998,88855859,bwadwellrq,EC2 Systems Engineer,L9,14:00,Wednesday,0,0,0,1


In [14]:
cand_hire_drop = interviewee_hiring.drop(columns=['Candidate_Name', 'Candidate_ICMS_ID', 'Last_Updated', 'Type'])
cand_hire_drop

,Interview_ID,Role,Level,Preferred_Time,Preferred_Day
0,51092998,EC2 Systems Engineer,L9,15:00,Tuesday
1,99241636,Professional Services Consultant,L8,13:00,Monday
2,64517874,EC2 Systems Engineer,L6,14:00,Thursday
3,75309134,Machine Learning Engineer,L6,12:00,Tuesday
4,31410407,Cloud Architect,L6,16:00,Wednesday
...,...,...,...,...,...
995,79716724,Delivery Practice Manager,L5,12:00,Monday
996,36886778,Marketing Manager,L5,10:00,Friday
997,51097937,Security Engineer,L9,11:00,Monday
998,88855859,Software Engineer,L8,13:00,Friday


In [15]:
voop_data_drop = voop_data.drop(columns=['Candidate_Name', 'Last_Updated'])
voop_data_drop

,Interview_ID,Employee_Alias,Date,Time,Decision,Duration,Level,Interview_Type,Communication_Rating,Leadership_Rating,Technical_Rating,Interest_Rating
0,36977658,ldelepine0,4/11/22,13:00,Strong No Hire,45,L9,Bar Raiser,4,5,6,2
1,67484217,aharral1,11/3/21,9:00,Inclined,45,L8,Loop,6,5,6,1
2,84630967,sdryburgh2,10/27/21,12:00,Inclined,60,L6,Phone Screen,3,5,2,6
3,9888727,chugonet3,5/26/22,12:00,Inclined,90,L6,Phone Screen,2,2,3,6
4,60055003,cfawthrop4,5/13/22,15:00,Strong No Hire,30,L6,Loop,6,3,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...
995,14556751,tvedntyevrn,8/20/21,17:00,Strongly Inclined,30,L5,Bar Raiser,3,1,4,4
996,83377630,afrakero,8/21/21,14:00,Strong No Hire,90,L5,Loop,1,4,5,6
997,40462641,ttilleyrp,8/4/21,10:00,Not Inclined,30,L9,Loop,3,6,3,2
998,78822341,bwadwellrq,1/19/22,9:00,Not Inclined,90,L8,Phone Screen,4,5,3,6


# Data Preprocessing for Model

In [16]:
pd.get_dummies(interviewer_hiring, columns=['Level', 'Role'], prefix='Emp')

,Interview_ID,Employee_Alias,Interviewer_Name,Candidate_Name,Candidate_ICMS_ID,Type,Preferred_Time,Preferred_Day,Last_Updated,Preferred_Day_Match,...,Emp_Operations Manager,Emp_Product Manager,Emp_Professional Services Consultant,Emp_Program Manager,Emp_Security Engineer,Emp_Software Development Engineer,Emp_Software Engineer,Emp_Solutions Architect,Emp_Supply Chain Manager,Emp_Technical Account Manager
0,51092998,ldelepine0,Linea Delepine,Agnese Normanton,7203708,Contract,10:00,Thursday,2021-08-20T23:11:27Z,0,...,0,0,0,0,0,0,0,0,0,0
1,99241636,aharral1,Arielle Harral,Davey Abrams,15918070,Part-time,14:00,Wednesday,2022-02-21T06:47:12Z,0,...,0,0,0,0,0,1,0,0,0,0
2,64517874,sdryburgh2,Shayne Dryburgh,Marrissa Langhor,67232318,Full-time,14:00,Thursday,2021-12-28T15:00:16Z,1,...,0,0,0,0,0,0,0,0,0,0
3,75309134,chugonet3,Claudetta Hugonet,Aurora Ireland,82904801,Contract,16:00,Monday,2022-05-10T13:58:46Z,0,...,0,0,0,0,1,0,0,0,0,0
4,31410407,cfawthrop4,Cyrille Fawthrop,Debora Greenall,76196522,Part-time,12:00,Friday,2022-03-14T11:33:48Z,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,79716724,tvedntyevrn,Terrie Vedntyev,Burgess Baumer,39023047,Part-time,16:00,Tuesday,2021-06-21T06:27:20Z,0,...,0,0,0,0,0,0,0,0,0,0
996,36886778,afrakero,Alvan Frake,Jocelyn Howman,40070522,Internship,14:00,Wednesday,2022-05-26T18:57:19Z,0,...,0,0,0,0,0,0,0,1,0,0
997,51097937,ttilleyrp,Torry Tilley,Ignacio Chivers,40720089,Contract,14:00,Tuesday,2021-07-10T01:44:33Z,0,...,0,1,0,0,0,0,0,0,0,0
998,88855859,bwadwellrq,Bryn Wadwell,Stillman Sproson,74769886,Part-time,14:00,Wednesday,2022-05-23T06:58:31Z,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
#df.reindex(columns=[list]) to reorder column names for data preprocessing.

In [19]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(master_df, y, test_size=0.30, random_state=101)

# Joins for Model

In [20]:
hiring_joined = pd.merge(int_hire_drop, cand_hire_drop, how='left', left_on=['Interview_ID'], right_on = ['Interview_ID'])
hiring_joined

,Interview_ID,Employee_Alias,Role_x,Level_x,Preferred_Time_x,Preferred_Day_x,Preferred_Day_Match,Preferred_Time_Match,Role_Match,Accepted_or_Not?,Role_y,Level_y,Preferred_Time_y,Preferred_Day_y
0,51092998,ldelepine0,Delivery Practice Manager,L4,10:00,Thursday,0,0,0,0,EC2 Systems Engineer,L9,15:00,Tuesday
1,99241636,aharral1,Software Development Engineer,L7,14:00,Wednesday,0,0,0,1,Professional Services Consultant,L8,13:00,Monday
2,64517874,sdryburgh2,Marketing Manager,L6,14:00,Thursday,1,1,0,0,EC2 Systems Engineer,L6,14:00,Thursday
3,75309134,chugonet3,Security Engineer,L9,16:00,Monday,0,0,0,0,Machine Learning Engineer,L6,12:00,Tuesday
4,31410407,cfawthrop4,Cloud Support Engineer,L5,12:00,Friday,0,0,0,0,Cloud Architect,L6,16:00,Wednesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,79716724,tvedntyevrn,Data Scientist,L7,16:00,Tuesday,0,0,0,0,Delivery Practice Manager,L5,12:00,Monday
996,36886778,afrakero,Solutions Architect,L8,14:00,Wednesday,0,0,0,0,Marketing Manager,L5,10:00,Friday
997,51097937,ttilleyrp,Product Manager,L4,14:00,Tuesday,0,0,0,1,Security Engineer,L9,11:00,Monday
998,88855859,bwadwellrq,EC2 Systems Engineer,L9,14:00,Wednesday,0,0,0,1,Software Engineer,L8,13:00,Friday


In [97]:
hire_joined_levels = hiring_joined.rename(columns={'Level_x':'Employee_Level', 'Level_y':'Candidate_Level'})
hire_joined_levels

,Interview_ID,Employee_Alias,Role_x,Employee_Level,Preferred_Time_x,Preferred_Day_x,Preferred_Day_Match,Preferred_Time_Match,Role_Match,Accepted_or_Not?,Role_y,Candidate_Level,Preferred_Time_y,Preferred_Day_y
0,51092998,ldelepine0,Delivery Practice Manager,L4,10:00,Thursday,0,0,0,0,EC2 Systems Engineer,L9,15:00,Tuesday
1,99241636,aharral1,Software Development Engineer,L7,14:00,Wednesday,0,0,0,1,Professional Services Consultant,L8,13:00,Monday
2,64517874,sdryburgh2,Marketing Manager,L6,14:00,Thursday,1,1,0,0,EC2 Systems Engineer,L6,14:00,Thursday
3,75309134,chugonet3,Security Engineer,L9,16:00,Monday,0,0,0,0,Machine Learning Engineer,L6,12:00,Tuesday
4,31410407,cfawthrop4,Cloud Support Engineer,L5,12:00,Friday,0,0,0,0,Cloud Architect,L6,16:00,Wednesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,79716724,tvedntyevrn,Data Scientist,L7,16:00,Tuesday,0,0,0,0,Delivery Practice Manager,L5,12:00,Monday
996,36886778,afrakero,Solutions Architect,L8,14:00,Wednesday,0,0,0,0,Marketing Manager,L5,10:00,Friday
997,51097937,ttilleyrp,Product Manager,L4,14:00,Tuesday,0,0,0,1,Security Engineer,L9,11:00,Monday
998,88855859,bwadwellrq,EC2 Systems Engineer,L9,14:00,Wednesday,0,0,0,1,Software Engineer,L8,13:00,Friday


In [21]:
test_voop = pd.get_dummies(hiring_joined, columns=['Level_x', 'Level_y'], prefix=['Emp','Cand'])
test_voop

,Interview_ID,Employee_Alias,Role_x,Preferred_Time_x,Preferred_Day_x,Preferred_Day_Match,Preferred_Time_Match,Role_Match,Accepted_or_Not?,Role_y,...,Emp_L6,Emp_L7,Emp_L8,Emp_L9,Cand_L4,Cand_L5,Cand_L6,Cand_L7,Cand_L8,Cand_L9
0,51092998,ldelepine0,Delivery Practice Manager,10:00,Thursday,0,0,0,0,EC2 Systems Engineer,...,0,0,0,0,0,0,0,0,0,1
1,99241636,aharral1,Software Development Engineer,14:00,Wednesday,0,0,0,1,Professional Services Consultant,...,0,1,0,0,0,0,0,0,1,0
2,64517874,sdryburgh2,Marketing Manager,14:00,Thursday,1,1,0,0,EC2 Systems Engineer,...,1,0,0,0,0,0,1,0,0,0
3,75309134,chugonet3,Security Engineer,16:00,Monday,0,0,0,0,Machine Learning Engineer,...,0,0,0,1,0,0,1,0,0,0
4,31410407,cfawthrop4,Cloud Support Engineer,12:00,Friday,0,0,0,0,Cloud Architect,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,79716724,tvedntyevrn,Data Scientist,16:00,Tuesday,0,0,0,0,Delivery Practice Manager,...,0,1,0,0,0,1,0,0,0,0
996,36886778,afrakero,Solutions Architect,14:00,Wednesday,0,0,0,0,Marketing Manager,...,0,0,1,0,0,1,0,0,0,0
997,51097937,ttilleyrp,Product Manager,14:00,Tuesday,0,0,0,1,Security Engineer,...,0,0,0,0,0,0,0,0,0,1
998,88855859,bwadwellrq,EC2 Systems Engineer,14:00,Wednesday,0,0,0,1,Software Engineer,...,0,0,0,1,0,0,0,0,1,0


# Adding Features

Here I feature engineered columns from the interviewer_hiring and interviewer_hiring tables to use as features for the model:

In [8]:
interviewer_hiring['Preferred_Day_Match'] = np.where(interviewer_hiring['Preferred_Day'] == interviewee_hiring['Preferred_Day'], 1, 0)
interviewer_hiring

,Interview_ID,Employee_Alias,Interviewer_Name,Candidate_Name,Candidate_ICMS_ID,Role,Type,Level,Preferred_Time,Preferred_Day,Last_Updated,Preferred_Day_Match
0,51092998,ldelepine0,Linea Delepine,Agnese Normanton,7203708,Delivery Practice Manager,Contract,L4,10:00,Thursday,2021-08-20T23:11:27Z,0
1,99241636,aharral1,Arielle Harral,Davey Abrams,15918070,Software Development Engineer,Part-time,L7,14:00,Wednesday,2022-02-21T06:47:12Z,0
2,64517874,sdryburgh2,Shayne Dryburgh,Marrissa Langhor,67232318,Marketing Manager,Full-time,L6,14:00,Thursday,2021-12-28T15:00:16Z,1
3,75309134,chugonet3,Claudetta Hugonet,Aurora Ireland,82904801,Security Engineer,Contract,L9,16:00,Monday,2022-05-10T13:58:46Z,0
4,31410407,cfawthrop4,Cyrille Fawthrop,Debora Greenall,76196522,Cloud Support Engineer,Part-time,L5,12:00,Friday,2022-03-14T11:33:48Z,0
...,...,...,...,...,...,...,...,...,...,...,...,...
995,79716724,tvedntyevrn,Terrie Vedntyev,Burgess Baumer,39023047,Data Scientist,Part-time,L7,16:00,Tuesday,2021-06-21T06:27:20Z,0
996,36886778,afrakero,Alvan Frake,Jocelyn Howman,40070522,Solutions Architect,Internship,L8,14:00,Wednesday,2022-05-26T18:57:19Z,0
997,51097937,ttilleyrp,Torry Tilley,Ignacio Chivers,40720089,Product Manager,Contract,L4,14:00,Tuesday,2021-07-10T01:44:33Z,0
998,88855859,bwadwellrq,Bryn Wadwell,Stillman Sproson,74769886,EC2 Systems Engineer,Part-time,L9,14:00,Wednesday,2022-05-23T06:58:31Z,0


In [136]:
interviewer_hiring['Preferred_Time_Match'] = np.where(interviewer_hiring['Preferred_Time'] == interviewee_hiring['Preferred_Time'], 1, 0)
interviewer_hiring.head()

,Interview_ID,Employee_Alias,Interviewer_Name,Candidate_Name,Candidate_ICMS_ID,Role,Type,Level,Preferred_Time,Preferred_Day,Last_Updated,Preferred_Day_Match,Preferred_Time_Match,Role_Match,Accepted_or_Not?
0,51092998,ldelepine0,Linea Delepine,Agnese Normanton,7203708,Delivery Practice Manager,Contract,L4,10:00,Thursday,2021-08-20T23:11:27Z,0,0,0,1
1,99241636,aharral1,Arielle Harral,Davey Abrams,15918070,Software Development Engineer,Part-time,L7,14:00,Wednesday,2022-02-21T06:47:12Z,0,0,0,1
2,64517874,sdryburgh2,Shayne Dryburgh,Marrissa Langhor,67232318,Marketing Manager,Full-time,L6,14:00,Thursday,2021-12-28T15:00:16Z,1,1,0,1
3,75309134,chugonet3,Claudetta Hugonet,Aurora Ireland,82904801,Security Engineer,Contract,L9,16:00,Monday,2022-05-10T13:58:46Z,0,0,0,1
4,31410407,cfawthrop4,Cyrille Fawthrop,Debora Greenall,76196522,Cloud Support Engineer,Part-time,L5,12:00,Friday,2022-03-14T11:33:48Z,0,0,0,1


In [137]:
interviewer_hiring['Role_Match'] = np.where(interviewer_hiring['Role'] == interviewee_hiring['Role'], 1, 0)
interviewer_hiring.head()

,Interview_ID,Employee_Alias,Interviewer_Name,Candidate_Name,Candidate_ICMS_ID,Role,Type,Level,Preferred_Time,Preferred_Day,Last_Updated,Preferred_Day_Match,Preferred_Time_Match,Role_Match,Accepted_or_Not?
0,51092998,ldelepine0,Linea Delepine,Agnese Normanton,7203708,Delivery Practice Manager,Contract,L4,10:00,Thursday,2021-08-20T23:11:27Z,0,0,0,1
1,99241636,aharral1,Arielle Harral,Davey Abrams,15918070,Software Development Engineer,Part-time,L7,14:00,Wednesday,2022-02-21T06:47:12Z,0,0,0,1
2,64517874,sdryburgh2,Shayne Dryburgh,Marrissa Langhor,67232318,Marketing Manager,Full-time,L6,14:00,Thursday,2021-12-28T15:00:16Z,1,1,0,1
3,75309134,chugonet3,Claudetta Hugonet,Aurora Ireland,82904801,Security Engineer,Contract,L9,16:00,Monday,2022-05-10T13:58:46Z,0,0,0,1
4,31410407,cfawthrop4,Cyrille Fawthrop,Debora Greenall,76196522,Cloud Support Engineer,Part-time,L5,12:00,Friday,2022-03-14T11:33:48Z,0,0,0,1


In [118]:
np.random.seed(42)
#interviewer_hiring['Accepted_or_Not?'] = #np.random.randint(0,2, size=len(interviewer_hiring))
interviewer_hiring['Accepted_or_Not?'] = np.where(interviewer_hiring['Preferred_Time_Match']== interviewer_hiring['Preferred_Day_Match'], 1,0)

#interviewer_hiring.apply(lambda x: x['Column1'] if x['Preferred_Time_Match'] == 1 then 1 else 0, axis=1)
interviewer_hiring

,Interview_ID,Employee_Alias,Interviewer_Name,Candidate_Name,Candidate_ICMS_ID,Role,Type,Level,Preferred_Time,Preferred_Day,Last_Updated,Preferred_Day_Match,Preferred_Time_Match,Role_Match,Accepted_or_Not?
0,51092998,ldelepine0,Linea Delepine,Agnese Normanton,7203708,Delivery Practice Manager,Contract,L4,10:00,Thursday,2021-08-20T23:11:27Z,0,0,0,1
1,99241636,aharral1,Arielle Harral,Davey Abrams,15918070,Software Development Engineer,Part-time,L7,14:00,Wednesday,2022-02-21T06:47:12Z,0,0,0,1
2,64517874,sdryburgh2,Shayne Dryburgh,Marrissa Langhor,67232318,Marketing Manager,Full-time,L6,14:00,Thursday,2021-12-28T15:00:16Z,1,1,0,1
3,75309134,chugonet3,Claudetta Hugonet,Aurora Ireland,82904801,Security Engineer,Contract,L9,16:00,Monday,2022-05-10T13:58:46Z,0,0,0,1
4,31410407,cfawthrop4,Cyrille Fawthrop,Debora Greenall,76196522,Cloud Support Engineer,Part-time,L5,12:00,Friday,2022-03-14T11:33:48Z,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,79716724,tvedntyevrn,Terrie Vedntyev,Burgess Baumer,39023047,Data Scientist,Part-time,L7,16:00,Tuesday,2021-06-21T06:27:20Z,0,0,0,1
996,36886778,afrakero,Alvan Frake,Jocelyn Howman,40070522,Solutions Architect,Internship,L8,14:00,Wednesday,2022-05-26T18:57:19Z,0,0,0,1
997,51097937,ttilleyrp,Torry Tilley,Ignacio Chivers,40720089,Product Manager,Contract,L4,14:00,Tuesday,2021-07-10T01:44:33Z,0,0,0,1
998,88855859,bwadwellrq,Bryn Wadwell,Stillman Sproson,74769886,EC2 Systems Engineer,Part-time,L9,14:00,Wednesday,2022-05-23T06:58:31Z,0,0,0,1


In [166]:
print(master_df_new.columns.tolist())

['hircand_Interview_ID', 'hircand_Candidate_Name', 'hircand_Candidate_ICMS_ID', 'hircand_Role', 'hircand_Type', 'hircand_Level', 'hircand_Preferred_Time', 'hircand_Preferred_Day', 'hircand_Last_Updated', 'hiremp_Interview_ID', 'hiremp_Employee_Alias', 'hiremp_Interviewer_Name', 'hiremp_Candidate_Name', 'hiremp_Candidate_ICMS_ID', 'hiremp_Role', 'hiremp_Type', 'hiremp_Level', 'hiremp_Preferred_Time', 'hiremp_Preferred_Day', 'hiremp_Last_Updated', 'voop_interview_id', 'voop_employee_alias', 'voop_date', 'voop_time', 'voop_candidate_name', 'voop_decision', 'voop_duration', 'voop_level', 'voop_interview_type', 'voop_communication_rating', 'voop_leadership_rating', 'voop_technical_rating', 'voop_interest_rating', 'voop_last_update', 'sched_employee_alias', 'sched_timestamp', 'sched_week_number', 'sched_slot_hour', 'sched_modified', 'sched_last_updated', 'ingestdate', 'emp_alias', 'emp_name', 'emp_skill', 'emp_skill_rating', 'emp_last_updated']


# Model to be Used:

In [151]:
training_model = hire_joined_levels[['Interview_ID', 'Accepted_or_Not?', 'Preferred_Day_Match', 'Preferred_Time_Match', 'Role_Match']]
training_model

,Interview_ID,Accepted_or_Not?,Preferred_Day_Match,Preferred_Time_Match,Role_Match
0,51092998,0,0,0,0
1,99241636,1,0,0,0
2,64517874,0,1,1,0
3,75309134,0,0,0,0
4,31410407,0,0,0,0
...,...,...,...,...,...
995,79716724,0,0,0,0
996,36886778,0,0,0,0
997,51097937,1,0,0,0
998,88855859,1,0,0,0


In [168]:
master_updated = master_df_new.merge(training_model, left_on='hiremp_Interview_ID', right_on='Interview_ID')
master_updated

,hircand_Interview_ID,hircand_Candidate_Name,hircand_Candidate_ICMS_ID,hircand_Role,hircand_Type,hircand_Level,hircand_Preferred_Time,hircand_Preferred_Day,hircand_Last_Updated,hiremp_Interview_ID,...,emp_alias,emp_name,emp_skill,emp_skill_rating,emp_last_updated,Interview_ID,Accepted_or_Not?,Preferred_Day_Match,Preferred_Time_Match,Role_Match
0,51092998,Agnese Normanton,7203708,EC2 Systems Engineer,Contract,L9,15:00,Tuesday,2022-02-04T16:18:37Z,51092998,...,ldelepine0,Linea Delepine,Compute and HPC - EC2 (Elastic Compute Cloud),1,2021-12-27T11:35:02Z,51092998,0,0,0,0
1,99241636,Davey Abrams,15918070,Professional Services Consultant,Contract,L8,13:00,Monday,2022-03-20T01:04:55Z,99241636,...,aharral1,Arielle Harral,Database - RDS SQL Server,5,2021-12-29T20:03:24Z,99241636,1,0,0,0
2,64517874,Marrissa Langhor,67232318,EC2 Systems Engineer,Part-time,L6,14:00,Thursday,2022-03-18T12:56:21Z,64517874,...,sdryburgh2,Shayne Dryburgh,ML/AI - Rekognition,1,2021-06-25T07:12:05Z,64517874,0,1,1,0
3,75309134,Aurora Ireland,82904801,Machine Learning Engineer,Internship,L6,12:00,Tuesday,2021-11-26T16:06:23Z,75309134,...,chugonet3,Claudetta Hugonet,Analytics - Lake Formation,3,2021-12-03T04:03:39Z,75309134,0,0,0,0
4,31410407,Debora Greenall,76196522,Cloud Architect,Full-time,L6,16:00,Wednesday,2022-02-20T12:24:35Z,31410407,...,cfawthrop4,Cyrille Fawthrop,ML/AI - Forecast,1,2022-03-20T08:45:19Z,31410407,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,79716724,Burgess Baumer,39023047,Delivery Practice Manager,Part-time,L5,12:00,Monday,2022-05-08T22:34:48Z,79716724,...,tvedntyevrn,Terrie Vedntyev,ML/AI - Forecast,5,2022-04-29T00:23:01Z,79716724,0,0,0,0
996,36886778,Jocelyn Howman,40070522,Marketing Manager,Full-time,L5,10:00,Friday,2021-07-01T16:28:56Z,36886778,...,afrakero,Alvan Frake,ML/AI - SageMaker,1,2022-05-02T03:46:12Z,36886778,0,0,0,0
997,51097937,Ignacio Chivers,40720089,Security Engineer,Internship,L9,11:00,Monday,2022-04-22T09:03:40Z,51097937,...,ttilleyrp,Torry Tilley,Compute and HPC - EC2 (Elastic Compute Cloud),4,2022-05-24T05:06:30Z,51097937,1,0,0,0
998,88855859,Stillman Sproson,74769886,Software Engineer,Contract,L8,13:00,Friday,2022-05-19T20:51:46Z,88855859,...,bwadwellrq,Bryn Wadwell,Analytics - Lake Formation,4,2022-06-11T22:42:18Z,88855859,1,0,0,0


In [170]:
master_updated.to_csv(r'C:\Users\sslavich\Desktop\export_dataframe.csv', index=False, header=True)

In [ ]:
master_select = master_updated

In [148]:
training_model.dtypes

Accepted_or_Not?         int64
Preferred_Day_Match      int64
Preferred_Time_Match     int64
Role_Match               int64
Employee_Level          object
Candidate_Level         object
dtype: object

In [102]:
training_model.to_csv(r'C:\Users\sslavich\Desktop\export_dataframe.csv', index=False, header=True)

# Invoking the Endpoint

To get an inference from a specific combination of features from the deployed model, we invoke the endpoint.

In [138]:
import json 
import boto3 
client = boto3.client('runtime.sagemaker')
 
data = {"instances": [0,0,0,'L4','L9']} 
response = client.invoke_endpoint(EndpointName='training-job-sean2-2c3dc83c285f4b709abe548abc2c99fc',
                                  Body=json.dumps(data), ContentType='text/csv')
response_body = response['Body'] 
print(response_body.read())

b'0,0.5095875859260559\n'


In [ ]:
aws sagemaker-runtime invoke-endpoint \
  --endpoint-name <endpoint-name> \
  --body '{"instances": [{"in0":[863],"in1":[882]}]}' \
  --content-type application/json \
  --accept application/json \
  results

# Basic Model

In [39]:
voop_data_drop

,Interview_ID,Employee_Alias,Date,Time,Decision,Duration,Level,Interview_Type,Communication_Rating,Leadership_Rating,Technical_Rating,Interest_Rating,Accepted or Not?
0,36977658,ldelepine0,4/11/22,13:00,Strong No Hire,45,L9,Bar Raiser,4,5,6,2,0
1,67484217,aharral1,11/3/21,9:00,Inclined,45,L8,Loop,6,5,6,1,1
2,84630967,sdryburgh2,10/27/21,12:00,Inclined,60,L6,Phone Screen,3,5,2,6,1
3,9888727,chugonet3,5/26/22,12:00,Inclined,90,L6,Phone Screen,2,2,3,6,1
4,60055003,cfawthrop4,5/13/22,15:00,Strong No Hire,30,L6,Loop,6,3,3,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,14556751,tvedntyevrn,8/20/21,17:00,Strongly Inclined,30,L5,Bar Raiser,3,1,4,4,0
996,83377630,afrakero,8/21/21,14:00,Strong No Hire,90,L5,Loop,1,4,5,6,0
997,40462641,ttilleyrp,8/4/21,10:00,Not Inclined,30,L9,Loop,3,6,3,2,0
998,78822341,bwadwellrq,1/19/22,9:00,Not Inclined,90,L8,Phone Screen,4,5,3,6,1


In [46]:
test_voop = pd.get_dummies(voop_data_drop, columns=['Level'])
test_voop

,Interview_ID,Employee_Alias,Date,Time,Decision,Duration,Interview_Type,Communication_Rating,Leadership_Rating,Technical_Rating,Interest_Rating,Accepted or Not?,Level_L4,Level_L5,Level_L6,Level_L7,Level_L8,Level_L9
0,36977658,ldelepine0,4/11/22,13:00,Strong No Hire,45,Bar Raiser,4,5,6,2,0,0,0,0,0,0,1
1,67484217,aharral1,11/3/21,9:00,Inclined,45,Loop,6,5,6,1,1,0,0,0,0,1,0
2,84630967,sdryburgh2,10/27/21,12:00,Inclined,60,Phone Screen,3,5,2,6,1,0,0,1,0,0,0
3,9888727,chugonet3,5/26/22,12:00,Inclined,90,Phone Screen,2,2,3,6,1,0,0,1,0,0,0
4,60055003,cfawthrop4,5/13/22,15:00,Strong No Hire,30,Loop,6,3,3,1,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,14556751,tvedntyevrn,8/20/21,17:00,Strongly Inclined,30,Bar Raiser,3,1,4,4,0,0,1,0,0,0,0
996,83377630,afrakero,8/21/21,14:00,Strong No Hire,90,Loop,1,4,5,6,0,0,1,0,0,0,0
997,40462641,ttilleyrp,8/4/21,10:00,Not Inclined,30,Loop,3,6,3,2,0,0,0,0,0,0,1
998,78822341,bwadwellrq,1/19/22,9:00,Not Inclined,90,Phone Screen,4,5,3,6,1,0,0,0,0,1,0


In [47]:
new_voop = test_voop[['Accepted or Not?', 'Level_L4', 'Level_L5', 'Level_L6', 'Level_L7', 'Level_L8', 'Level_L9']]
new_voop

,Accepted or Not?,Level_L4,Level_L5,Level_L6,Level_L7,Level_L8,Level_L9
0,0,0,0,0,0,0,1
1,1,0,0,0,0,1,0
2,1,0,0,1,0,0,0
3,1,0,0,1,0,0,0
4,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...
995,0,0,1,0,0,0,0
996,0,0,1,0,0,0,0
997,0,0,0,0,0,0,1
998,1,0,0,0,0,1,0


In [48]:
new_voop.to_csv(r'C:\Users\sslavich\Desktop\export_dataframe.csv', index=False, header=True)